In [1]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('brown')
from nltk.corpus import brown, stopwords
from nltk import sent_tokenize, word_tokenize
from string import punctuation
import string

[nltk_data] Downloading package brown to /home/vinicius/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
nltk.download('webtext')
from nltk.corpus import webtext

[nltk_data] Downloading package webtext to /home/vinicius/nltk_data...
[nltk_data]   Package webtext is already up-to-date!


In [3]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

In [4]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import nltk

[nltk_data] Downloading package punkt to /home/vinicius/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vinicius/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/vinicius/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
stopwords_en = set(stopwords.words('english'))
stopwords_en_withpunct = stopwords_en.union(set(punctuation))

In [6]:
stopwords_json = {"en":["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]}
stopwords_json_en = set(stopwords_json['en'])
stopwords_nltk_en = set(stopwords.words('english'))
stopwords_punct = set(punctuation)
# Combine the stopwords. Its a lot longer so I'm not printing it out...
stoplist_combined = set.union(stopwords_json_en, stopwords_nltk_en, stopwords_punct)

In [7]:
train = pd.read_json('./data/train.json')
test = pd.read_json('./data/test.json')

In [8]:
## Rename text and target columns
train.rename(columns={
    'request_id':'id',
    'request_text_edit_aware':'text',
    'request_title':'title',
    'requester_received_pizza':'target'
},inplace=True)

## Rename text columns
test.rename(columns={
    'request_id':'id',
    'request_text_edit_aware':'text',
    'request_title':'title'  
},inplace=True)


In [9]:
## Get only text columns
train = train[['id','text','title','target']]
test = test[['id','text','title']]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 

In [11]:
Xtrain, Xval, ytrain, yval = train_test_split(train.drop('target',axis=1), train['target'])

In [12]:
def preprocess_text(text):
    # Input: str, i.e. document/sentence
    # Output: list(str) , i.e. list of lemmas
    return [word for word in lemmatize_sent(text) 
            if word not in stoplist_combined
            and not word.isdigit()]

In [13]:
def lemmatize_sent(text):
    wnl = WordNetLemmatizer()
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]

In [14]:
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' # if mapping isn't found, fall back to Noun.

In [15]:
## Count Vectorizer to train and Validation Set
count_vect = CountVectorizer(analyzer=preprocess_text)

In [16]:
train_set = count_vect.fit_transform(Xtrain['text'])

In [17]:
val_set = count_vect.transform(Xval['text'])

In [18]:
## Count VEctorizer to full dataset (more data)
vectorizer = CountVectorizer(analyzer=preprocess_text)

In [19]:
data = vectorizer.fit_transform(train['text'])

In [20]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB() 
# To train the classifier, simple do 
clf.fit(train_set, ytrain) 

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
from sklearn.metrics import accuracy_score

predictions_valid = clf.predict(val_set)

print('Pizza reception accuracy = {}'.format(
        accuracy_score(predictions_valid, yval) * 100)
     )

Pizza reception accuracy = 74.05940594059406


In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
mdl = RandomForestClassifier(n_estimators=500, random_state=0,n_jobs=-1)

In [24]:
mdl.fit(train_set,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [25]:
pred_RF = mdl.predict(val_set)

print('Pizza reception accuracy = {}'.format(
        accuracy_score(pred_RF, yval) * 100)
     )

Pizza reception accuracy = 75.44554455445545


## Model for all DATA

In [26]:
## Train Random Forest for all data
mdl.fit(data,train['target'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [28]:
test_set = vectorizer.transform(test['text'])

In [29]:
pred = mdl.predict(test_set)

In [41]:
test['target'] = pred.astype('int')

In [42]:
df_sub = test[['id','target']]

In [49]:
df_sub.rename(columns={
    'id':'request_id',
    'target':'requester_received_pizza'
},inplace=True)

/home/vinicius/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [56]:
df_sub.to_csv('new_sub',index=False)

In [55]:
sample = pd.read_csv('./data/sampleSubmission.csv')

In [53]:
sample

,request_id,requester_received_pizza
0,t3_i8iy4,0
1,t3_1mfqi0,0
2,t3_lclka,0
3,t3_1jdgdj,0
4,t3_t2qt4,0
...,...,...
1626,t3_knttk,0
1627,t3_11wza2,0
1628,t3_iwbsf,0
1629,t3_nys7g,0
